In [ ]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv", index_col = 'id')
df_test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv", index_col = 'id')
sample_submission = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")

FEATURES = df.columns[:-1]
TARGET = df.columns[-1]

In [ ]:
df_test.head()

In [ ]:
df.shape

To check whether there are null values in train set (df)

In [ ]:
print(df.isnull().sum())

there seems to be null values so we need to use method to come over this. 

In [ ]:
y = df[TARGET]
X= df.drop(TARGET, axis = 1)
X_test = df_test.copy()
X_test.head()

In [ ]:
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
print(numerical_cols)

In [ ]:
print(len(df.columns))
print(len(X.columns))

Spliting the training and target in training and validation 

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y, 
                    train_size = 0.8, test_size = 0.2, random_state =0)

There are no cateforical columns and therefore the focus will on the numerical columns and apply simple imputer method to handle the null values

In [ ]:
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))




In [ ]:
# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
imputed_X_test.columns = X_test.columns

In [ ]:
imputed_X_train.head()

The use of XGBoost Classifier Model because its an classification problem. 

In [ ]:
#model = RandomForestRegressor(n_estimators=100, random_state=0, verbose = 1000)
#model_xgboost = XGBClassifier(random_state = 42, tree_method= 'gpu_hist', gpu_id = 0, predictor='gpu_predictor')

model_xgboost= XGBClassifier(max_depth=3, subsample=0.5, colsample_bytree=0.5, n_jobs=-1,
    # Uncomment if you want to use GPU. Recommended for whole training set.
    tree_method='gpu_hist',gpu_id = 0, predictor='gpu_predictor',
    random_state=0,
)

In [ ]:

model_xgboost.fit(imputed_X_train, y_train, verbose = False,
             eval_set = [(X_train, y_train), (X_valid, y_valid)],
             eval_metric = "auc",
             early_stopping_rounds = 200)
preds = model_xgboost.predict_proba(X_valid)[:,1]
score = roc_auc_score(y_valid, preds)
print('Score: ', score)

In [ ]:
X_test.head()

In [ ]:
#test_preds = model_xgboost.predict_proba(imputed_X_test)[:,1]

test_preds = pd.Series(
    model_xgboost.predict(imputed_X_test),
    index=X_test.index,
    name=TARGET,
)

In [ ]:
test_preds

In [ ]:
test_preds.to_csv("submission.csv")